In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.13.1

In [0]:
%matplotlib inline

import os
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, GlobalAveragePooling3D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils.vis_utils import model_to_dot

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
from google.colab import drive
drive.mount('./gdrive')

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [3]:
%cd "gdrive/My Drive/Colab Notebooks/data"

/content/gdrive/My Drive/Colab Notebooks/data


In [0]:
def Shortcut3D(input_data, fx, data_format = "channels_first"):

  # チャンネル数の取得
  if data_format == "channels_first":
    channel_num = int(fx.shape[1])
  else:
    channel_num = int(fx.shape[4])
      
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  buf = Convolution3D(channel_num, (1,1,1), strides=(1,1,1), padding='valid', data_format=data_format)(input_data)

  # 2つを足す
  return Add()([buf, fx])

In [0]:
def Resblock3D(n_filters, strides=(1,1,1), data_format = "channels_first", axis=1):
  def f(input_data):   
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
    fx = BatchNormalization(axis=axis)(fx)
    fx = Activation('relu')(fx)
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(fx)
    fx = BatchNormalization(axis=axis)(fx)

    return Shortcut3D(input_data, fx, data_format)

  return f

In [0]:
def Resnet3D(channel = 3, frame = 10, height = 64, width = 64, num_classes = 2, is_channels_first = True):
  if is_channels_first:
    data_format = "channels_first"
    axis=1
    input_data = Input(shape=(channel, frame, height, width))
  else:
    data_format = "channels_last"
    axis=-1
    input_data = Input(shape=(frame, height, width, channel))
  
  print(input_data.shape)

  x = Convolution3D(32, (7,7,7), strides=(1,1,1), kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
  x = BatchNormalization(axis=axis)(x)
  x = Activation('relu')(x)
  x = MaxPooling3D((3, 3,3), strides=(2,2,2), padding='same', data_format=data_format)(x)


  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = MaxPooling3D(strides=(2,2,2), data_format=data_format)(x)  
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)


  x =  GlobalAveragePooling3D(data_format=data_format)(x)
  x = Dense(num_classes, kernel_initializer='he_normal', activation='softmax')(x)

  model = Model(inputs=input_data, outputs=x)
  return model

In [7]:
is_channels_first = False

if is_channels_first:
  print("A")
  buf = np.load('./resnet3d_data_channel_first_updown_and_wipe_20191009.npz')


else:
  print("B")
  buf = np.load('./resnet3d_data_channel_last_updown_and_wipe_20191010.npz')

X = buf["X"].astype("f2")
Y = buf["Y"]
del buf

Y = keras.utils.to_categorical(Y, 2, dtype="f2").astype("f2")
X_train, X_val,  Y_train,  Y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.8, shuffle=True)

B


In [0]:
def step_decay(epoch):
    x = 0.1
    if epoch >= 2: x /= 5.0
    if epoch >= 4: x /= 5.0
    if epoch >= 6: x /= 5.0
    return x
  
decay = LearningRateScheduler(step_decay, verbose=1)

In [8]:
model = Resnet3D(channel=3, frame=10, height=200, width=200, num_classes=2, is_channels_first=is_channels_first)
adam = keras.optimizers.Adam()
#lr_decay = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.0001 + 0.02 * np.math.pow(0.5, 1+epoch), verbose=True)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath=os.path.join("./", 'ResNet34-3D_epoch-{epoch:02d}_{val_loss:.2f}.hdf5'), save_best_only=True)

(?, 10, 200, 200, 3)
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
is_tpu = True

if is_tpu:
  # TPU
  tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  strategy = tf.contrib.tpu.TPUDistributionStrategy(tpu_cluster_resolver)
  model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)

INFO:tensorflow:Querying Tensorflow master (grpc://10.21.50.82:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6030402138270382769)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9377350620376703158)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1232352298506557261)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10769092011599936229)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7356597634080261761)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/j

In [0]:
history = model.fit(X_train, Y_train,
                    batch_size=128,
                    epochs=200,
                    verbose=1,
                    # callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1)],
                    callbacks=[checkpoint],
                    validation_data=(X_test, Y_test))

Train on 2329 samples, validate on 583 samples
Epoch 1/200
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 10, 200, 200, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 2), dtype=tf.float32, name='dense_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for input_1
